In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### 의약품 제품 허가 정보

허가정보 1단계 접속 URL

end_point = 'http://apis.data.go.kr/1471000/DrugPrdtPrmsnInfoService02'

인증키

service_key='i4kidk21c3X/hLkySwIiPJ8aTr1xRqQ/CWnluHr1zG2jx1LUBZrsfheuojj5ZlO79XhfL0gAEfmrz3TqpVWidg=='

허가목록 정보

registered = 'getDrugPrdtPrmsnInfoService02'

허가목록 상세정보

detailed = 'getDrugPrdtPrmsnDtlInq01'

허가 주성분 상세정보

ingredients = 'getDrugPrdtMcpnDtlInq'

In [2]:
# 소독제 품목기준코드 기준으로 추출
item_seqs = pd.read_csv("disin_basic/disinfectants.csv")['품목명']

In [3]:
item_seqs.head(5)

0    신형개인제독킷(군납명:제독키트,피부/개인장비용,KD-1)(군납용)
1                        한국루베디에스투제독액(군납용)
2                    싸이덱스오피에이액(오토프탈알데하이드)
3                        스테리스-20액(35%과초산)
4                     아세사이드액(저농도과초산평형혼합물)
Name: 품목명, dtype: object

In [4]:
#항목 parsing 함수작성하기
def parse():
    try:
        ENTRPS_PRMISN_NO = item.find("ENTRPS_PRMISN_NO").get_text()
        ENTRPS = item.find("ENTRPS").get_text()
        PRDUCT = item.find("PRDUCT").get_text()
        MTRAL_SN = item.find("MTRAL_SN").get_text()
        MTRAL_CODE = item.find("MTRAL_CODE").get_text()
        MTRAL_NM = item.find("MTRAL_NM").get_text()
        QNT = item.find("QNT").get_text()
        ITEM_SEQ = item.find("ITEM_SEQ").get_text()

        return {
            "업체허가번호":ENTRPS_PRMISN_NO,
            "업체명":ENTRPS,
            "제품명":PRDUCT,
            "일련번호":MTRAL_SN,
            "원료코드":MTRAL_CODE,
            "원료명":MTRAL_NM,
            "분량":QNT,
            "품목기준코드":ITEM_SEQ
             }
    except AttributeError as e:
        return {
            "업체허가번호":None,
            "업체명":None,
            "제품명":None,
            "일련번호":None,
            "원료코드":None,
            "원료명":None,
            "분량":None,
            "품목기준코드":None
             }

In [5]:
# 의약품 제품 허가정보 추출
service_key = 'i4kidk21c3X/hLkySwIiPJ8aTr1xRqQ/CWnluHr1zG2jx1LUBZrsfheuojj5ZlO79XhfL0gAEfmrz3TqpVWidg=='
url = 'http://apis.data.go.kr/1471000/DrugPrdtPrmsnInfoService02/getDrugPrdtMcpnDtlInq?'

In [6]:
test = []

for n in item_seqs:
    param = {'serviceKey' : service_key, 
              'pageNo' : '1', 
              'numOfRows' : '1',
              'Prduct' : n
            }
    
    r = requests.get(url, param)
    soup = BeautifulSoup(r.text, 'lxml-xml')
    items = soup.find_all("item")
    
    # row = []
    # row2 = []
    for item in items:
        test.append(parse())

In [7]:
df_2 = pd.DataFrame(test)
print(df_2.shape)
df_2.head(5)

(22, 8)


,업체허가번호,업체명,제품명,일련번호,원료코드,원료명,분량,품목기준코드
0,1596,삼양화학공업(주),"신형개인제독킷(군납명:제독키트,피부/개인장비용,KD-1)(군납용)",1,M092244,음이온수지분말,3.38,199803487
1,2029,(주)한국루베,한국루베디에스투제독액(군납용),1,M082676,디에칠렌트리아민,70,199804680
2,876,에이에스피코리아(유),싸이덱스오피에이액(오토프탈알데하이드),1,M091762,O-프탈알데하이드,0.55,200008757
3,443,(주)사라야코리아,아세사이드액(저농도과초산평형혼합물),1,M102203,저농도과초산평형혼합물,100,200511464
4,1338,(주)퍼슨,오피덱스액(오토프탈알데하이드),1,M091762,O-프탈알데하이드,0.55,201201533


In [8]:
df_2[df_2['제품명']=="에이치엠씨엔에프산(과탄산나트륨)"]

,업체허가번호,업체명,제품명,일련번호,원료코드,원료명,분량,품목기준코드
6,366,(주)엠에이치헬스케어,에이치엠씨엔에프산(과탄산나트륨),1,M095079,과탄산나트륨,370,201601386


In [9]:
df_2.to_csv("disin_active.csv", mode="w")

In [12]:
#item_seq = ['199704546', '199704566', '200109042', '201306157', '201501433','199503356','199600444','200005554','198701194','198701191','201403766','200600800','200802231','200103999','200404293','200804324','201507982','199906930','201702267','200511464','200801715','200008757','201601386','201209564','201708248','200700624','201201533','201606718','201607103','201700721','201700813','201701117','201901806']

#참고사항으로 pageNo 기준값은 520

row = []
for n in item_seqs:
    params = {'serviceKey' : service_key, 
              'pageNo' : '1', 
              'numOfRows' : '100',
              'item_seq' : n
            }
    
    r = requests.get(url, params)
    soup = BeautifulSoup(r.text, 'lxml-xml')
    items = soup.find_all("item")
    
    # row = []
    # row2 = []
    for item in items:
        row.append(parse())
    
#    row2 = []
#    for r in row:
#        row2.append(r)

In [4]:
df_it = pd.DataFrame(row)
df_it.shape

(52006, 18)

In [5]:
df_it.tail(10)

,품목기준코드,품목명,업체명,품목허가일자,업종,품목일련번호,전문/일반,분류명,품목허가번호,주성분,주성분수,신고/허가,취하일자,취하구분,이미지,업일련번호,업허가번호,보험코드
51996,202203487,디에논정(디에노게스트),주식회사 더유제약,20220906,의약품,202203487,전문의약품,[02470]난포호르몬제 및 황체호르몬제,170,디에노게스트,1,허가,,정상,,20182522,373,
51997,202203490,에어젠의료용아산화질소,에어젠,20220906,의약품,202203490,전문의약품,[01110]전신마취제,4,아산화질소,1,신고,,정상,,2021003110,107,
51998,202203492,핀쥬베스프레이(피나스테리드),(주)보령,20220906,의약품 및 의약외품 수입업,202203492,전문의약품,"[02670]모발용제(발모, 탈모, 염모, 양모제)",5021,피나스테리드,1,허가,,정상,,20161741,249,
51999,202203495,트리탄정150밀리그램(트리메부틴말레산염),맥널티제약(주),20220906,의약품,202203495,일반의약품,[02390]기타의 소화기관용약,530,트리메부틴말레산염,1,신고,,정상,,19710001,1524,
52000,202203496,데오란트센스액(염화알루미늄수화물),태극제약(주),20220906,의약품,202203496,일반의약품,"[01250]발한제,지한제",5227,염화알루미늄수화물,1,신고,,정상,,19580005,1370,
52001,202203501,엠로딘액(L-카르니틴),맥널티제약(주),20220906,의약품,202203501,일반의약품,[02190]기타의 순환계용약,531,L-카르니틴,1,신고,,정상,,19710001,1524,
52002,202203503,브이페낙점안액(브롬페낙나트륨수화물)(1회용),한미약품(주),20220907,의약품,202203503,전문의약품,[01310]안과용제,5289,브롬페낙나트륨수화물,1,허가,,정상,,19570027,1337,
52003,202203504,"올타딘점안액0.7%(올로파타딘염산염)(1회용), 올타딘점안액0.7%(올로파타딘염산염)",풍림무약(주),20220907,의약품,202203504,전문의약품,[01310]안과용제,5165,올로파타딘염산염/올로파타딘염산염,2,허가,,정상,,19670005,1480,
52004,202203508,한림디에노게스트정2밀리그램,한림제약(주),20220907,의약품,202203508,전문의약품,[02470]난포호르몬제 및 황체호르몬제,5213,디에노게스트,1,허가,,정상,,19690009,1496,
52005,202203513,모드콜콜드시럽,(주)종근당,20220907,의약품,202203513,일반의약품,[01140]해열.진통.소염제,5349,dl-메틸에페드린염산염/덱스트로메토르판브롬화수소산염수화물/아세트아미노펜/클로르페니라...,4,신고,,정상,,19650001,1459,


In [6]:
df_it['업체명'].unique()

array(['(주)종근당', '제이더블유중외제약(주)', '엔비케이제약(주)', '대한약품공업(주)', '(주)제일제약',
       '동화약품(주)', '경남제약(주)', '대원제약(주)', '일동제약(주)', '신신제약(주)', '삼진제약(주)',
       '정우신약(주)', '일심제약', '서울약품공업(주)', '(주)유한양행', '영진약품(주)',
       '에이치케이이노엔(주)', '아주약품(주)', '부광약품(주)', '서울약품(주)', '광동제약(주)', '(주)한독',
       '삼남제약(주)', '(주)씨엠지제약', '원광제약(주)', '동아에스티(주)', '지엘파마(주)', '동아제약(주)',
       '(주)유유제약', '(주)보령', '일양약품(주)', '한국원자력연구원', '영풍제약(주)', '삼아제약(주)',
       '(주)대웅제약', '삼일제약(주)', '안국약품(주)', '환인제약(주)', '일성신약(주)',
       '에스케이플라즈마(주)', '(주)휴온스', '(주)넥스팜코리아', '하나제약(주)', '동광제약(주)',
       '에이프로젠제약(주)', '제일약품(주)', '동성제약(주)', '천우신약(주)', '영일제약(주)',
       '천혜당제약(주)', '국제약품(주)', '(주)퍼슨', '익수제약(주)', '신풍제약(주)', '(유)한풍제약',
       '한올바이오파마(주)', '대일화학공업(주)', '신일제약(주)', '(주)녹십자', '동국제약(주)',
       '(주)바이넥스', '알보젠코리아(주)', '(주)동구바이오제약', '한미약품(주)', '구주제약(주)',
       '현대약품(주)', '한국유나이티드제약(주)', '(주)한국신약', '경동제약(주)', '알파제약(주)',
       '(주)하원제약', '(주)유영제약', '대우제약(주)', '한화제약(주)', '삼익제약(주)', '이연제약(주)',
       '신화제약(주)', '(주)마더스제약', '태극제약

In [7]:
df_it[df_it['업체명']=='(주)에이치피앤씨']

,품목기준코드,품목명,업체명,품목허가일자,업종,품목일련번호,전문/일반,분류명,품목허가번호,주성분,주성분수,신고/허가,취하일자,취하구분,이미지,업일련번호,업허가번호,보험코드
20338,200801715,페라스텔액(과초산4%),(주)에이치피앤씨,20080213,의약품 및 의약외품 수입업,200801715,전문의약품,[07390]기타의 공중위생용약,1,과초산 4%,1,허가,,정상,,20162735,472,
41530,201901806,트리스텔 스포리와입스앤폼,(주)에이치피앤씨,20190326,의약품 및 의약외품 수입업,201901806,전문의약품,[07390]기타의 공중위생용약,7,시트르산수화물/아염소산나트륨(25%),2,허가,,정상,,20162735,472,
49913,202103785,트리스텔듀오폼,(주)에이치피앤씨,20210511,의약품 및 의약외품 수입업,202103785,전문의약품,[07390]기타의 공중위생용약,8,시트르산수화물/아염소산나트륨25%,2,허가,,정상,,20162735,472,
50780,202107876,페라스텔멀티샷액(과아세트산4%),(주)에이치피앤씨,20211210,의약품,202107876,전문의약품,[07390]기타의 공중위생용약,1,과아세트산4%,1,신고,,정상,,2021016415,6170,


In [8]:
disinfectants=df_it[df_it['분류명']=='[07390]기타의 공중위생용약']

In [9]:
print(disinfectants.shape)
disinfectants

(37, 18)


,품목기준코드,품목명,업체명,품목허가일자,업종,품목일련번호,전문/일반,분류명,품목허가번호,주성분,주성분수,신고/허가,취하일자,취하구분,이미지,업일련번호,업허가번호,보험코드
6551,199803487,"신형개인제독킷(군납명:제독키트,피부/개인장비용,KD-1)(군납용)",삼양화학공업(주),19981105,의약품,199803487,전문의약품,[07390]기타의 공중위생용약,7,분말활성탄/양이온수지분말/음이온수지분말,3,허가,,정상,,19800003,1596,
6592,199804680,한국루베디에스투제독액(군납용),(주)한국루베,19981111,의약품,199804680,일반의약품,[07390]기타의 공중위생용약,1,디에칠렌트리아민/수산화나트륨,2,허가,,정상,,19980016,2029,
8990,200008757,싸이덱스오피에이액(오토프탈알데하이드),에이에스피코리아(유),20001204,의약품 및 의약외품 수입업,200008757,전문의약품,[07390]기타의 공중위생용약,1,O-프탈알데하이드,1,허가,,정상,,20207865,876,
9941,200109042,스테리스-20액(35%과초산),(주)테크노랜드,20010206,의약품 및 의약외품 수입업,200109042,전문의약품,[07390]기타의 공중위생용약,2,35%과초산,1,허가,20210731,폐업,,20162978,564,
16013,200511464,아세사이드액(저농도과초산평형혼합물),(주)사라야코리아,20050319,의약품 및 의약외품 수입업,200511464,일반의약품,[07390]기타의 공중위생용약,2,저농도과초산평형혼합물,1,허가,,정상,,20162665,443,
20338,200801715,페라스텔액(과초산4%),(주)에이치피앤씨,20080213,의약품 및 의약외품 수입업,200801715,전문의약품,[07390]기타의 공중위생용약,1,과초산 4%,1,허가,,정상,,20162735,472,
25822,201201533,오피덱스액(오토프탈알데하이드),(주)퍼슨,20120309,의약품,201201533,전문의약품,[07390]기타의 공중위생용약,5006,O-프탈알데하이드,1,허가,,정상,,19570002,1338,
31237,201501433,스테리스-40액(35.5%과아세트산),(주)테크노랜드,20150318,의약품 및 의약외품 수입업,201501433,전문의약품,[07390]기타의 공중위생용약,3,35.5%과아세트산,1,허가,20210731,폐업,,20162978,564,
33440,201507982,시트로린액(시트르산수화물),(주)휴온스메디텍,20151127,의약품,201507982,전문의약품,[07390]기타의 공중위생용약,140,시트르산수화물,1,허가,,정상,,20020013,2286,
34045,201601386,에이치엠씨엔에프산(과탄산나트륨),(주)엠에이치헬스케어,20160309,의약품 및 의약외품 수입업,201601386,일반의약품,[07390]기타의 공중위생용약,4,과탄산나트륨,1,허가,,정상,,20162367,366,


In [11]:
disinfectants.to_csv("disinfectants.csv", mode="w")

In [79]:
row2 = []

for r in row:
    row2.append(parse())

In [83]:
df_it.head()

,품목기준코드,품목명,업체명,품목허가일자,업종,품목일련번호,전문/일반,분류명,품목허가번호,주성분,주성분수,신고/허가,취하일자,취하구분,이미지,업일련번호,업허가번호,보험코드
0,202203478,오스틴구풍해독탕연조엑스,오스틴제약(주),20220905,의약품,202203478,일반의약품,[01320]이비과용제,5126,감초/강활/길경/방풍/석고/연교/우방자/형개,8,신고,,정상,,19590006,1376,
1,202203478,오스틴구풍해독탕연조엑스,오스틴제약(주),20220905,의약품,202203478,일반의약품,[01320]이비과용제,5126,감초/강활/길경/방풍/석고/연교/우방자/형개,8,신고,,정상,,19590006,1376,
2,202203478,오스틴구풍해독탕연조엑스,오스틴제약(주),20220905,의약품,202203478,일반의약품,[01320]이비과용제,5126,감초/강활/길경/방풍/석고/연교/우방자/형개,8,신고,,정상,,19590006,1376,
3,202203478,오스틴구풍해독탕연조엑스,오스틴제약(주),20220905,의약품,202203478,일반의약품,[01320]이비과용제,5126,감초/강활/길경/방풍/석고/연교/우방자/형개,8,신고,,정상,,19590006,1376,
4,202203478,오스틴구풍해독탕연조엑스,오스틴제약(주),20220905,의약품,202203478,일반의약품,[01320]이비과용제,5126,감초/강활/길경/방풍/석고/연교/우방자/형개,8,신고,,정상,,19590006,1376,


In [75]:
r = requests.get(url, params)
soup = BeautifulSoup(r.text, 'lxml-xml')
items = soup.find_all("item")

In [76]:
row = []

for item in items:
    row.append(parse())

In [77]:
df=pd.DataFrame(row)
print(df.shape)
df.head()

(0, 0)


""


In [ ]:
print(items)

In [ ]:
df=pd.DataFrame(row)
df.head(1)

In [ ]:
df.to_csv("medical_registered.csv", mode='w',)

In [ ]:
r_json = r.json

In [ ]:
print(r_json)

In [ ]:
df = pd.read_json(r.text)

In [ ]:
df.head()

In [ ]:
# 허가정보 1단계 접속 URL
end_point = 'http://apis.data.go.kr/1471000/DrugPrdtPrmsnInfoService02'

# 인증키
service_key = 'i4kidk21c3X/hLkySwIiPJ8aTr1xRqQ/CWnluHr1zG2jx1LUBZrsfheuojj5ZlO79XhfL0gAEfmrz3TqpVWidg=='

# 허가목록 정보
registered = 'DrugPrdtPrmsnInfoService02'

# 허가목록 상세정보
detailed = 'getDrugPrdtPrmsnDtlInq01'

# 허가 주성분 상세정보
ingredients = 'getDrugPrdtMcpnDtlInq'

In [ ]:
# 허가품목 기본정보 추출
url = end_point + '/' + registered +"?{}".format(service_key)
url

In [ ]:
content = requests.get(url).content
print(content)

In [ ]:
params ={'pageNo' : '1', 
         'numOfRows' : '10', 
         'type' : 'json', 
         'item_name' : '', 
         'entp_name' : '', 
         'item_permit_date' : '', 
         'entp_no' : '', 
         'bar_code' : '', 
         'item_seq' : '', 
         'start_change_date' : '', 
         'end_change_date' : '', 
         'order' : '', 
         'edi_code' : '', 
         'atc_code' : ''}

의약품 소독제 ATC 코드 번호: V07AV (Technical disinfectants)


In [ ]:
params ='?' + 'serviceKey=' + service_key + '&type=' + 'xml' + '&pageNo=1' + '&numOfRows=10'

In [ ]:
web=url+params
web

In [ ]:
result = requests.get(url+params)
#result.content
bs_result = BeautifulSoup(result.content, 'html.parser')
print(bs_result)

In [ ]:
url = end_point + '/' + registered + '?'
params ={'serviceKey' : service_key, 
         'pageNo' : '1', 
         'numOfRows' : '10', 
         'type' : 'json', 
         'item_name' : '', 
         'entp_name' : '', 
         'item_permit_date' : '', 
         'entp_no' : '', 
         'bar_code' : '', 
         'item_seq' : '', 
         'start_change_date' : '', 
         'end_change_date' : '', 
         'order' : '', 
         'edi_code' : '', 
         'atc_code' : ''}

In [ ]:
r2 = requests.get(url, params=params)
r2.text

In [ ]:
r = requests.get(url, params=params)
r.text

In [ ]:
import json

In [ ]:
json=json(r)
json

In [ ]:
len(json)

In [ ]:
json[0]